In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv")

In [ ]:
cat = [col for col in train.columns if 'cat' in col]
cont = [col for col in train.columns if 'cont' in col]
all_features = cat + cont

In [ ]:
all_df = pd.concat([train , test]).reset_index(drop = True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat:
    all_df[col] = le.fit_transform(all_df[col])
    
train = all_df[:train.shape[0]]
test = all_df[train.shape[0]:].reset_index(drop = True)

In [ ]:
data = train[all_features]
target = train['target']

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import statistics

results, predictions = [], []
X_train, X_test, y_train, y_test = train_test_split(data , target , \
                                        test_size = 0.028059109276941666 , random_state = 42)
    
model = xgb.XGBClassifier(n_estimators=20000,max_depth=5,reg_lambda=0.9, random_state=42)
    
model.fit(X_train, y_train, early_stopping_rounds=300, eval_set=[(X_test, y_test)], verbose=False)
y_prediction = model.predict(X_test)

results.append(accuracy_score(y_test,y_prediction))
    
print(results)

In [ ]:
y_prob = model.predict_proba(X_test)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_prob[:,1], average='micro'))

In [ ]:
y_prob